In [1]:
import pandas as pd
import numpy as np
from multiprocessing import Pool
from functools import partial
import Evaluation_func as func
from tqdm import tqdm_notebook as tqdm
from datetime import date, timedelta
import pickle

In [2]:
df = pd.read_csv(r'D:\Customer_Value\data\evaluate\st_sample_all_large.csv')

One Hot Encoding

In [3]:
df[['AMT_B_label', 'AMT_S_label', 'ST_ASSET_label', 'MP_ASSET_label', 'SS_ASSET_label', 'TRADE']] = df[['AMT_B_label', 'AMT_S_label', 'ST_ASSET_label', 'MP_ASSET_label', 'SS_ASSET_label', 'TRADE']].astype(int)
df = pd.get_dummies(df, columns=['AMT_B_label', 'AMT_S_label', 'ST_ASSET_label', 'MP_ASSET_label', 'SS_ASSET_label', 'TRADE'], prefix=['AMT_B', 'AMT_S', 'ST_ASSET', 'MP_ASSET', 'SS_ASSET', 'TRADE'])

In [4]:
for i in range(1, 11):
    for col in ['ST_ASSET', 'MP_ASSET', 'SS_ASSET']:
        if f'{col}_{-1 * i}' not in df.columns:
            df[f'{col}_{-1 * i}'] = 0

Transfer Data to Local Training

In [5]:
df_list = [group[1] for group in df.groupby(df['sample_no'])]
output = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(func.transfer_data, df_list), total=len(df_list)), 1):
            output.append(x)

Save Local Training Data

In [6]:
with open(r'D:\Customer_Value\data\evaluate\local\evaluate_large', 'wb') as fp:
    pickle.dump(output, fp)

Transfer Data to Google Training

In [3]:
df_list = [group[1] for group in df.groupby(df['sample_no'])]
output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(func.transfer_google, df_list), total=len(df_list)), 1):
            output_list.append(x)
            
df = pd.concat(output_list, axis=0)

In [14]:
df_list = [group[1] for group in df.groupby(df['sample_no'])]
output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(func.transfer_google_feature, df_list), total=len(df_list)), 1):
            output_list.append(x)
            
df = pd.concat(output_list, axis=0)

In [15]:
df['GENDER'] = df['GENDER'].apply(lambda x: 1 if x == 'M' else 0)
df['LABEL_CHURN'] = df['LABEL_CHURN'].apply(lambda x: 1 if x == 'CHURN' else 0)

In [16]:
test = pd.read_csv(r'D:\Customer_Value\data\train_google\test_feature.csv')
test_churn = test[test.LABEL_CHURN == 1]

In [17]:
test = pd.concat([df, test_churn], axis=0)

Save Data

In [19]:
test.to_csv(r'D:\Customer_Value\data\evaluate\google\test_eval_feature.csv', index=False)

In [1]:
test

NameError: name 'test' is not defined